In [1]:
import numpy as np
from cardiacmap.data import CascadeDataFile
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import butter, filtfilt
import cv2


In [2]:
CascadeDataFile = CascadeDataFile.load_data("2011-08-23_Exp000_Rec112_Cam1-Blue.dat")
signal = CascadeDataFile.signals[0]

In [3]:
def normalize(data):

    return ((data - np.amin(data)) / (np.amax(data) - np.amin(data)) * 256).astype(np.uint8)

    
def normalize_flat(data):

    return ((data - np.amin(data, axis=0)) / (np.amax(data, axis=0) - np.amin(data, axis=0)) * 256).astype(np.uint8)

In [9]:
signal.normalize()


[[3574 3762 3905 ... 2275 2268 2270]
 [3604 3711 3760 ... 2262 2262 2260]
 [3500 3605 3635 ... 2258 2257 2250]
 ...
 [2392 2386 2393 ... 2259 2259 2256]
 [2369 2370 2378 ... 2250 2253 2248]
 [2155 2160 2159 ... 2157 2154 2156]]


In [4]:
signal.perform_average(type="time", sig=4, rad=3)
signal.perform_average(type="spatial", sig=8, rad=6)

(4657, 128, 128)


In [5]:
# Config 1
data = normalize(signal.base_data)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter('video_1.mp4', fourcc, 230, (128, 128), True)

for frame in data:
    writer.write(cv2.applyColorMap(frame, cv2.COLORMAP_RAINBOW))

writer.release()

In [7]:
import ffmpeg

In [6]:
def _render_video(data, output_filename):

    # Initialize ffmpeg input stream

    process = (
        ffmpeg.input('pipe:', format='rawvideo', pix_fmt='bgr24', s='128x128', r=230)
        .output(output_filename, vcodec='libx264', pix_fmt='yuv420p')
        .overwrite_output()
        .run_async(pipe_stdin=True)
    )

    for frame in data:

        color_mapped_frame = cv2.applyColorMap(frame, cv2.COLORMAP_RAINBOW)
        process.stdin.write(color_mapped_frame.astype(np.uint8).tobytes())

    process.stdin.close()

    process.wait()

    print(f"Video saved as {output_filename}")


In [9]:
_render_video(data, "test.mp4")

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Video saved as test.mp4


frame= 4657 fps=0.0 q=-1.0 Lsize=     184kB time=00:00:20.23 bitrate=  74.7kbits/s speed=30.5x    
video:129kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 42.866543%
[libx264 @ 0x55c676ec2200] frame I:19    Avg QP:22.92  size:  2426
[libx264 @ 0x55c676ec2200] frame P:1173  Avg QP:26.41  size:    33
[libx264 @ 0x55c676ec2200] frame B:3465  Avg QP:36.37  size:    13
[libx264 @ 0x55c676ec2200] consecutive B-frames:  0.8%  0.0%  0.0% 99.2%
[libx264 @ 0x55c676ec2200] mb I  I16..4:  2.4% 62.2% 35.4%
[libx264 @ 0x55c676ec2200] mb P  I16..4:  0.0%  0.0%  0.0%  P16..4:  6.0%  1.3%  0.5%  0.0%  0.0%    skip:92.2%
[libx264 @ 0x55c676ec2200] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8:  1.0%  0.0%  0.0%  direct: 0.0%  skip:99.0%  L0:14.2% L1:85.8% BI: 0.0%
[libx264 @ 0x55c676ec2200] 8x8 transform intra:62.2% inter:96.1%
[libx264 @ 0x55c676ec2200] coded y,uvDC,uvAC intra: 94.4% 99.8% 95.1% inter: 0.3% 1.1% 0.0%
[libx264 @ 0x55c676ec2200] i16 v,h,dc,p:  3%  7%  7% 83%
[l

In [20]:
# Config 2
data = normalize_flat(signal.base_data)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter('video_2.mp4', fourcc, 230, (128, 128), True)

for frame in data:
    writer.write(cv2.applyColorMap(frame, cv2.COLORMAP_RAINBOW))

writer.release()

In [21]:
# Config 3
data = normalize(signal.transformed_data)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter('video_3.mp4', fourcc, 230, (128, 128), True)

for frame in data:
    writer.write(cv2.applyColorMap(frame, cv2.COLORMAP_RAINBOW))

writer.release()

In [22]:
# Config 4
data = normalize_flat(signal.transformed_data)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter('video_4.mp4', fourcc, 230, (128, 128), True)

for frame in data:
    writer.write(cv2.applyColorMap(frame, cv2.COLORMAP_RAINBOW))

writer.release()